In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### XToYHTo2B2Tau UL production  -> Y(2B) X(2Tau)

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     processParameters = cms.vstring(
                                                     __CHANNEL_DECAY_FRAGMENT__
                                                     ),
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 'processParameters'
                                                                                 )
                                                     )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2016APV': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(True),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2016", "2016APV", "2017", "2018"]
mX = ["300", "400", "500", "550", "600", "650", "700", "800", "900", "1000", "1200", "1400", "1600", "1800", "2000", "2500", "3000", "3500", "4000"]
mY = ["60", "70", "80", "90", "95", "100", "125", "150", "200", "300", "400", "500", "600", "800", "1000", "1200", "1400", "1600", "1800", "2000", "2600", "3000", "3500"]

tot_events1 = [400000]*len(mY)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/portalesHEP/genproductions/tree/8268afa49656697442cf771d359f964a52b641e1/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/NMSSM_XToYH'
    t_datasetname_year = 'NMSSM_XToYHTo2B2ZTo2B4L_MX-{massX}_MY-{massY}_TuneCP5_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/NMSSM_XYH/NMSSM_XToYH_MX_{massX}_MY_{massY}/v1/NMSSM_XToYH_MX_{massX}_MY_{massY}_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    
    dataset_names_year = []
    gp_ggf_year = []

    for mx in mX:
        for my in mY:
            d = {"massX": mx, "massY": my}
            tmp_example_dict = t_example1
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict = dataset_names_year
            tmp_gridpack_dict = gp_ggf_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

#print(dataset_names['2017'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

In [ ]:
x_hy_bb4l = '''                                                                                                                                                                                           
    # Force Higgs decays
    '25:onMode = off',                     # Turn off all default H decays
    '25:oneChannel = 1 1 100 23 23',       # H → Z Z
    '25:onIfMatch = 23 23',

    # Force Z to leptons only (e, μ only)
    '23:onMode = off',                     # Turn off all default Z decays
    '23:onIfAny = 11 13',                  # Z → e⁺e⁻ or μ⁺μ⁻ only

    # Force Y → bb~
    '35:onMode = off',
    '35:oneChannel = 1 1 100 5 -5',
    '35:onIfMatch = 5 -5',

    # Resonance decay filter for exact final state: bb ℓℓℓℓ
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on',
    'ResonanceDecayFilter:eMuTauAsEquivalent = on',
    'ResonanceDecayFilter:allNuAsEquivalent  = off',  # not needed
    'ResonanceDecayFilter:udscAsEquivalent  = off',   # not needed
    'ResonanceDecayFilter:mothers = 35,25,23',
    'ResonanceDecayFilter:daughters = 5,5,11,11,13,13'  # bb eeμμ (ℓℓℓℓ)
'''
#'SpaceShower:dipoleRecoil = on' 


## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator(x_hy_bb4l)
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('XtoYHto2B4L' + '_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        '''if year == "2016":
                if process == "Radion":
                    pr = 1606
                elif process == "BulkGraviton":
                    pr = 1631
            if year == "2016APV":
                if process == "Radion":
                    pr = 812
                elif process == "BulkGraviton":
                    pr = 837
            if year == "2017":
                if process == "Radion":
                    pr = 1574
                elif process == "BulkGraviton":
                    pr = 1599
            if year == "2018":
                if process == "Radion":
                    pr = 1612
                elif process == "BulkGraviton":
                    pr = 1637'''
        k=0
        for i in range(len(mX)):
            for j in range(len(mY)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.6.5"
                dataset_name = dataset_names[year][k]
                if year == "2016APV":
                    events = round(tot_events1[j]*0.54)
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGENAPV-00' + str(pr)
                elif year == "2016":
                    events = round(tot_events1[j]*0.46)
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                elif year == "2017":
                    events = tot_events1[j]
                    #prepid = 'HIG-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                elif year == "2018":
                    events = tot_events1[j]
                    #prepid = 'HIG-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + "  Pythia8"
                mcdb_id = '0'
                time = '0.01'
                size = '30'
                if int(mX[i]) > (int(mY[j]) + 125):
                    csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                #pr = pr+1   
                k=k+1